In [ ]:
import pandas as pd
import requests
import json

In [ ]:
cisa_df = pd.read_json("https://www.cisa.gov/sites/default/files/feeds/known_exploited_vulnerabilities.json")
cisa_df = pd.json_normalize(cisa_df['vulnerabilities'])
cisa_df = cisa_df.rename(columns={'cveID': 'CVE'})


In [ ]:
cisa_df =  cisa_df
cves = cisa_df["CVE"].to_list()
cves

CISA_List = []

for cve in cves:
    base_url = 'https://cve.circl.lu/api/cve/'
    url = ''.join([base_url, cve])
    r = requests.get(url)
    try:
        data = r.json()
        CVEPublished = data['Published']
        CVEModified = data['Modified']
        cvss = data['cvss']
        cwe = data['cwe']
        references = data['references']
        vulnerable_product = data['vulnerable_product']

    except:
        pass
    CISA_List_Temp = (cve, CVEPublished, CVEModified, cvss, cwe, references, vulnerable_product)
    CISA_List.append(CISA_List_Temp)

In [ ]:
cisa_enriched_df = pd.DataFrame(CISA_List, columns = ["CVE", "CVE_Published", "CVE_Modified", "CVSS", "CWE", "References", "Vulnerable_Products"])

In [ ]:
cisa_data = cisa_enriched_df.merge(cisa_df, on='CVE', how='outer')
cisa_data  = cisa_data.rename(columns={'dueDate': 'CISA_Due_Date', 'requiredAction' : 'Required_Action', 'shortDescription' : "Description", 'vendorProject': 'Vendor', 'product' : 'Product', 'vulnerabilityName' : 'Vulnerability_Name', 'dateAdded' : 'CISA_Date_Added'})
cisa_data= cisa_data[[ 'CVE', 'Vulnerability_Name', 'Description', 'Vendor', 'Product', 'CVSS', 'CWE', 'Required_Action', 'CVE_Published', 'CVE_Modified', 'CISA_Date_Added', 'CISA_Due_Date', 'References', 'Vulnerable_Products']]

In [ ]:

cisa_data.to_csv('cisa_enriched.csv', index=False)

In [ ]:
result =  cisa_data.to_json(orient="records")
parsed = json.loads(result)
print(json.dumps(parsed, indent=4))

with open('cisa_enriched.json', 'w', encoding='utf-8') as f:
    json.dump(parsed, f, ensure_ascii=False, indent=4)
